In [4]:
import os
import random

# 设置调整因子 k，根据实际数据调整这个值
k = 500  # 需要根据数据的规模和分布进行调整

# 总文件夹路径，其中包含各个领域的子文件夹
root_folder_path = '/data3/zhouqiang/LLM_scratch/dataset/COIG-CQIA'
domain_data_counts = {}
total_data_count = 0

# 计算每个领域的数据总量
for domain in os.listdir(root_folder_path):
    domain_path = os.path.join(root_folder_path, domain)
    if os.path.isdir(domain_path):
        jsonl_files = [f for f in os.listdir(domain_path) if f.endswith('.jsonl')]
        domain_total = 0
        for jsonl_file in jsonl_files:
            file_path = os.path.join(domain_path, jsonl_file)
            with open(file_path, 'r', encoding='utf-8') as file:
                lines_count = sum(1 for line in file)
                domain_total += lines_count
        domain_data_counts[domain] = domain_total
        total_data_count += domain_total

# 新JSONL文件的路径
output_file_path = '/data3/zhouqiang/LLM_scratch/dataset/inverse_sampled_data.jsonl'
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

# 按逆比例采样并合并到一个新文件
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for domain, count in domain_data_counts.items():
        if count > 0:  # 检查以避免除以零
            sample_rate = k / count  # 计算逆比例采样率
            sample_rate = min(sample_rate, 1)  # 保证采样率不超过100%
            domain_path = os.path.join(root_folder_path, domain)
            jsonl_files = [f for f in os.listdir(domain_path) if f.endswith('.jsonl')]
            for jsonl_file in jsonl_files:
                file_path = os.path.join(domain_path, jsonl_file)
                with open(file_path, 'r', encoding='utf-8') as file:
                    lines = file.readlines()
                    sampled_lines = random.sample(lines, max(1, int(len(lines) * sample_rate)))
                    output_file.writelines(sampled_lines)
        else:
            print(f"No data to sample in domain: {domain}")

print(f"Inverse sampling completed and combined data is stored in: {output_file_path}")


No data to sample in domain: .git
Inverse sampling completed and combined data is stored in: /data3/zhouqiang/LLM_scratch/dataset/inverse_sampled_data.jsonl
